In [1]:
import pandas as pd
import os

In [3]:
tweets_csv_dir = os.getcwd() + '/data/tweets_csv/'
tweets_csv_dir

'/Users/akaashkambath/Akaash/gradSchool/firstTerm/w207/w207_finalproject/data/tweets_csv/'

In [5]:
april_1_2 = pd.read_csv('data/tweets_csv/april1_april2.csv')
april_1_2.head(3)

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified
0,"36.77943,-1.29889",Wed Apr 01 09:01:18 +0000 2020,corona coronavirus covid stayhome,NaN,https://www.instagram.com/p/B-brG4pFTdp/?igshi...,0,1245275030956544002,NaN,NaN,NaN,...,6723,3,15,"EAST AFRICA, WORLDWIDE",NDANAH MIQASSA,bandanahmiqassa,23042,NaN,https://youtu.be/0uu8AzxGmeI,False
1,"6.12745379,49.64968784",Wed Apr 01 08:17:02 +0000 2020,actufoot football luxembourg,NaN,http://dlvr.it/RSy1bQ,0,1245263893682769921,NaN,NaN,NaN,...,395,89,8,Luxembourg,FuPa Luxembourg,fupalux,6243,NaN,http://www.fupa.net/luxemburg,False
2,"-0.32863,51.2334699",Wed Apr 01 09:01:11 +0000 2020,homeschooling stayhome corona,NaN,https://www.instagram.com/p/B-brXf-hhna/?igshi...,0,1245275001563099136,NaN,NaN,NaN,...,79,193,0,NaN,Emilie-Kate Owen,emiliekateowen,236,NaN,NaN,False


In [6]:
april_1_2.columns

Index(['coordinates', 'created_at', 'hashtags', 'media', 'urls',
       'favorite_count', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'retweet_count', 'retweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url', 'user_created_at',
       'user_screen_name', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name', 'user_screen_name.1',
       'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified'],
      dtype='object')

In [12]:
import twarc